In [ ]:
import os
# os.chdir('../')
# os.getcwd()
%load_ext autoreload
%autoreload 2
import src.forecaster.diagnostic as diagnostic
import  src.forecaster.modelil as mod
import src.forecaster.utilitaires as util
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


raw_master = pd.read_csv('data/raw/raw_master_il_1019.csv')

fc_eln = pd.read_csv('data/raw/DI/di_forecast.csv')

In [ ]:
mod = mod.Modelil(raw_master)

max_date_available = mod.all_sales.calendar_yearmonth.max()
filter_date = min(201908, max_date_available)
dwps = util.create_list_period(201601, filter_date, False)
dwp, dtp = util.get_all_combination_date(dwps, 10)

print("creating the main table")
table_all_features = mod.create_all_features(dwp, dtp)

In [ ]:
from copy import deepcopy

XGBOOST_PARAMETERS = {
            'max_depth': 11,  # 25
            'gamma': 0.02,
            'subsample': 0.4,
            'n_estimators': 27,
            'learning_rate': 0.1,
            'n_jobs': 12,
            'verbosity': 2
        }
dico_params_eib = {}
for i in range(1,11):
    dico_params_eib[i] = XGBOOST_PARAMETERS.copy()

#dico_params[7]['n_estimators'] = 49
dico_params_eib[5]['n_estimators'] = 27 
dico_params_eib[6]['n_estimators'] = 45
dico_params_eib[7]['n_estimators'] = 31  
dico_params_eib[8]['n_estimators'] = 47 
dico_params_eib[9]['n_estimators'] = 13 
dico_params_eib[10]['n_estimators'] = 27

dico_params_di = deepcopy(dico_params_eib)
dico_params_di[6]['n_estimators'] = 81

In [ ]:
params={}
params['eib']= dico_params_eib
params['di'] = dico_params_di

In [ ]:
dwp_test = util.create_list_period(201702, 201902, False)

In [ ]:
res = mod.recreate_past_forecasts(table_all_features, dwp_test, params, max_available_date=201908)

In [ ]:
test = diagnostic.Diagnostic(cvr=res[(res.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)

# KPIS with atleast 2 years of data at date when predciting

In [ ]:
month_min=24 # minimal age in month at date when predicting
pred_horizon = 6
test = diagnostic.Diagnostic(cvr=res[(res.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6, MINIMUM_HISTO = month_min + pred_horizon)

In [ ]:
import numpy as np

fc_eln['cycle_month'] = pd.to_datetime(fc_eln['cycle_month'], format= "%Y-%m")
fc_eln['forecasted_month'] = pd.to_datetime(fc_eln['forecasted_month'], format= "%Y-%m")

fc_eln['horizon']  = 12 * (fc_eln['forecasted_month'].dt.year - fc_eln['cycle_month'].dt.year) + (fc_eln['forecasted_month'].dt.month - fc_eln['cycle_month'].dt.month)

fc_eln['horizon'] = fc_eln['horizon'].astype(int)

fc_eln['sku_wo_pkg'] = fc_eln.sku_w_pkg.str.split('_').str[0:3].str.join('_') + '_'+ fc_eln.sku_w_pkg.str.split('_').str[4]

fc_eln.horizon += 2
fc_eln['label'] = 'di'
fc_eln.rename(columns={'forecasted_month':'date_to_predict'}, inplace=True)
fc_eln.rename(columns={'cycle_month':'date_when_predicting'}, inplace=True)
fc_eln.rename(columns={'forecast':'prediction'}, inplace=True)
fc_eln['date_when_predicting'] = fc_eln['date_when_predicting'].dt.strftime('%Y%m')

fc_eln_di = fc_eln.groupby(['date_to_predict', 'sku_wo_pkg', 'horizon'])['prediction'].sum().reset_index()

tempdi = fc_eln_di.copy()
tempeib = res[res.label=='eib'].copy()

tempdi['date_to_predict'] = pd.to_datetime(tempdi['date_to_predict']).dt.year.astype(
            str) + pd.to_datetime(tempdi['date_to_predict']).dt.month.astype(str).str.zfill(2)
tempdi['date_to_predict'] = tempdi['date_to_predict'].astype(int)
tempdi['label'] = 'di'

In [ ]:
tempdi.rename(columns={'prediction': 'preddi'}, inplace=True)
tempeib.rename(columns={'prediction': 'predeib'}, inplace=True)
tempil = pd.merge(
tempeib[['date_to_predict', 'sku_wo_pkg', 'predeib', 'horizon']],
tempdi[['date_to_predict', 'sku_wo_pkg', 'preddi', 'horizon']],
how='left',
on=['date_to_predict', 'sku_wo_pkg', 'horizon']
)
tempdiaug = tempil.copy()
tempdiaug['preddi'] = tempil['preddi'].fillna(0)
tempdiaug['label']='di'
tempdi = tempdiaug
tempil['prediction'] = tempil['preddi'].fillna(0) + tempil['predeib'].fillna(0)

tempil['label'] = 'il'
tempdi.rename(columns={'preddi': 'prediction'}, inplace=True)
tempeib.rename(columns={'predeib': 'prediction'}, inplace=True)
rescombined= pd.concat([tempdi, tempeib, tempil], sort=True).drop(['preddi', 'predeib'], axis=1, errors="ignore")

# KPIs with DI preds replaced by ELN predicitons 

In [ ]:
test = diagnostic.Diagnostic(cvr=rescombined[(rescombined.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)

# BELOW KPIS MIXED AF AND ELN

In [ ]:
from src.forecaster.utilitaires import fa_score, fa_score_prime, bias_score, add_sku_age
master_il = add_sku_age(raw_master, 'date', ['sku_wo_pkg'],
                                         ['offtake_di', 'offtake_il', 'offtake_eib'],
                                         Thre_abs=12, Thre_rel=0.02)



cvr = test.ensure_output_format(res[(res.horizon==6)], True)

age = master_il[['date', 'sku_wo_pkg', 'country', 'brand', 'tier', 'stage', 'age_offtake_di',
                              'age_offtake_il', 'age_offtake_eib']][master_il.date >= cvr.date.min()]
age = age.rename(columns={'date': 'date'})
age['date'] = pd.to_datetime(age['date'], format='%Y%m')

cvr = cvr.merge(age, on=['date', 'sku_wo_pkg'], how='left', validate='m:1').fillna(-99)

In [ ]:
res_corrected = res.copy()

In [ ]:
cvr['date_to_predict'] = pd.to_datetime(cvr['date'], format='%Y-%m-%d')
cvr['date_to_predict'] = cvr['date_to_predict'].dt.strftime('%Y%m').astype(int)
res_corrected = pd.merge(res_corrected, cvr[['sku_wo_pkg', 'date_to_predict', 'age_offtake_di', 'horizon']], on = ['date_to_predict', 'horizon', 'sku_wo_pkg'] , how = 'left')

In [ ]:
tmp=res_corrected[(res_corrected['label']=='di')&(res_corrected.age_offtake_di>=30)]
tmp.rename(columns={'prediction' : 'pred2'},inplace=True)
tmp.date_to_predict = tmp.date_to_predict.astype(int)
final_res = pd.merge(rescombined, tmp[['sku_wo_pkg', 'date_to_predict', 'label', 'horizon', 'pred2']], how = 'left', on = ['sku_wo_pkg', 'date_to_predict', 'label', 'horizon'])

In [ ]:
final_res.loc[final_res.pred2>0, 'prediction'] = final_res.loc[final_res.pred2>0, 'pred2']

In [ ]:
test = diagnostic.Diagnostic(cvr=final_res[(final_res.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)

In [ ]:
tempdi = final_res[final_res.label == 'di']
tempeib = final_res[final_res.label == 'eib']
tempil = final_res[final_res.label == 'il']

tempdi.rename(columns={'prediction': 'preddi'}, inplace=True)
tempeib.rename(columns={'prediction': 'predeib'}, inplace=True)
tempil = pd.merge(
    tempdi[['date_to_predict', 'sku_wo_pkg', 'preddi', 'horizon']],
    tempeib[['date_to_predict', 'sku_wo_pkg', 'predeib', 'horizon']],
    how='left',
    on=['date_to_predict', 'sku_wo_pkg', 'horizon']
)
tempil['prediction'] = tempil['preddi'] + tempil['predeib']

tempil['label'] = 'il'
tempdi.rename(columns={'preddi': 'prediction'}, inplace=True)
tempeib.rename(columns={'predeib': 'prediction'}, inplace=True)

finafinal=pd.concat([tempdi, tempeib, tempil], sort=True).drop(['preddi', 'predeib'], axis=1, errors="ignore")

In [ ]:
test = diagnostic.Diagnostic(cvr=finafinal[(finafinal.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)